We are doing the followign here
- drop : cols high correlation
- manual dummies
- LOTS of different models with CROSS VALIDATION

In [1]:
#import the libraries:
import pandas as pd  

In [2]:
#import the dataset:
diamond = pd.read_csv("train.csv",index_col="id")

In [3]:
diamond = diamond.drop(columns = ['x','y','z']) #HIGH CORRELATION
diamond.head()

,carat,cut,color,clarity,depth,table,price
id,,,,,,,
0,0.50,Ideal,D,VS2,62.3,55.0,1845
1,1.54,Good,I,VS1,63.6,60.0,10164
2,1.32,Very Good,J,SI2,61.7,60.0,5513
3,1.20,Ideal,I,SI1,62.1,55.0,5174
4,1.73,Premium,I,SI1,61.2,60.0,10957


In [4]:
diamond["cut"].replace({"Ideal": 5, "Premium": 4, "Very Good" : 3, "Good" : 2, "Fair" : 1}, inplace=True)
diamond["color"].replace({"D": 7, "E": 6, "F" : 5, "G" : 4, "H" : 3, "I": 2 , "J": 1}, inplace=True)
diamond["clarity"].replace({"IF": 9, "VVS1": 8, "VVS2" : 7, 
                             "VS1" : 6, "VS2" : 5, "SI1": 4, 
                             "SI2" : 2, "I1": 1 }, inplace=True)

In [5]:
diamond.head()

,carat,cut,color,clarity,depth,table,price
id,,,,,,,
0,0.50,5,7,5,62.3,55.0,1845
1,1.54,2,2,6,63.6,60.0,10164
2,1.32,3,1,2,61.7,60.0,5513
3,1.20,5,2,4,62.1,55.0,5174
4,1.73,4,2,4,61.2,60.0,10957


In [6]:
diamond.dtypes

carat      float64
cut          int64
color        int64
clarity      int64
depth      float64
table      float64
price        int64
dtype: object

#### The data is now ready fot the different algorithms

# MODELS:

In [7]:
X = diamond.drop(columns = ['price'])
y = diamond['price'] 

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split( X, y, train_size=0.90)

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## 1. Decission tree

In [11]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier

In [12]:
tree = DecisionTreeClassifier(max_depth = 3)
tree.fit(X_train, y_train)
y_pred_train_tree = tree.predict(X_train)
y_pred_test_tree = tree.predict(X_test)

In [13]:
from sklearn import metrics
#TRAIN:
print('TRAIN : Mean Absolute Error:', metrics.mean_absolute_error(y_train, y_pred_train_tree))
print('TRAIN : Mean Squared Error:', metrics.mean_squared_error(y_train, y_pred_train_tree))
print('TRAIN : Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train_tree)))
print ('----------------------------------------------------------')
#TEST:
print('TEST : Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_test_tree ))
print('TEST : Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_test_tree ))
print('TEST : Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_test_tree )))

TRAIN : Mean Absolute Error: 3186.786096844187
TRAIN : Mean Squared Error: 25787192.580982722
TRAIN : Root Mean Squared Error: 5078.109154102806
----------------------------------------------------------
TEST : Mean Absolute Error: 3284.31586752348
TEST : Mean Squared Error: 26557366.507661887
TEST : Root Mean Squared Error: 5153.383986048574


In [14]:
#Lets corroborate these values with cross validation:
from sklearn.model_selection import cross_val_score
scores_tree = cross_val_score(tree, X_train, y_train, cv=3, scoring='neg_root_mean_squared_error')
scores_tree

/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


array([-5109.32445006, -5058.98641903, -4572.30149319])

In [15]:
# This value is VERY HIGH --------> NOT VALID

## 2. Random forest

In [64]:
from sklearn.ensemble import RandomForestRegressor

In [65]:
randforest = RandomForestRegressor(n_estimators=20, random_state=10)
randforest.fit(X_train, y_train)
y_pred_train_randforest = randforest.predict(X_train)
y_pred_test_randforest = randforest.predict(X_test)

In [66]:
from sklearn.metrics import r2_score

In [67]:
#Training set
r2_train = r2_score(y_pred = y_pred_train_randforest, y_true=y_train)
r2_train

0.9968199608936993

In [68]:
#Test set
r2_test = r2_score(y_pred = y_pred_test_randforest, y_true=y_test)
r2_test

0.9787108348941431

In [61]:
#TRAIN:
print('TRAIN : Mean Absolute Error:', metrics.mean_absolute_error(y_train, y_pred_train_randforest))
print('TRAIN : Mean Squared Error:', metrics.mean_squared_error(y_train, y_pred_train_randforest))
print('TRAIN : Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train_randforest)))
print ('----------------------------------------------------------')
#TEST:
print('TEST : Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_test_randforest ))
print('TEST : Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_test_randforest ))
print('TEST : Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_test_randforest )))

TRAIN : Mean Absolute Error: 116.10708468330905
TRAIN : Mean Squared Error: 51074.21562552794
TRAIN : Root Mean Squared Error: 225.99605223438735
----------------------------------------------------------
TEST : Mean Absolute Error: 299.4279016666998
TEST : Mean Squared Error: 345143.97274553136
TEST : Root Mean Squared Error: 587.4895511798753


In [62]:
#Lets corroborate these values with cross validation:
scores_randforest = cross_val_score(randforest, X_train, y_train, cv=3, scoring='neg_root_mean_squared_error')
scores_randforest

array([-579.25012055, -581.64985798, -574.69065574])

In [63]:
#It is less than 1.000 --------> VALID

## 3. K-nearest neighbor

In [24]:
from sklearn.neighbors import KNeighborsClassifier

In [25]:
neiclass = KNeighborsClassifier(n_neighbors=5)
neiclass.fit(X_train, y_train)

KNeighborsClassifier()

In [26]:
y_pred_train_neiclass = neiclass.predict(X_train)
y_pred_test_neiclass = neiclass.predict(X_test)

In [27]:
#TRAIN:
print('TRAIN : Mean Absolute Error:', metrics.mean_absolute_error(y_train, y_pred_train_neiclass))
print('TRAIN : Mean Squared Error:', metrics.mean_squared_error(y_train, y_pred_train_neiclass))
print('TRAIN : Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train_neiclass)))
print ('----------------------------------------------------------')
#TEST:
print('TEST : Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_test_neiclass ))
print('TEST : Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_test_neiclass ))
print('TEST : Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_test_neiclass )))

TRAIN : Mean Absolute Error: 761.6138592106347
TRAIN : Mean Squared Error: 1878595.038342168
TRAIN : Root Mean Squared Error: 1370.6184875238507
----------------------------------------------------------
TEST : Mean Absolute Error: 928.8509639149778
TEST : Mean Squared Error: 2370247.410034602
TEST : Root Mean Squared Error: 1539.560784780712


In [28]:
#Lets corroborate these values with cross validation:
scores_neighclass = cross_val_score(neiclass, X_train, y_train, cv=3, scoring='neg_root_mean_squared_error')
scores_neighclass

/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


array([-1640.89477042, -1621.57534831, -1630.6102287 ])

In [29]:
#It is more than 1.000 --------> NOT VALID

## 4. Bayesian Ridge

In [30]:
from sklearn.linear_model import BayesianRidge 

In [31]:
bay = BayesianRidge() 
bay.fit(X_train, y_train) 

BayesianRidge()

In [32]:
y_pred_train_bay = bay.predict(X_train)
y_pred_test_bay = bay.predict(X_test)

In [33]:
#TRAIN:
print('TRAIN : Mean Absolute Error:', metrics.mean_absolute_error(y_train, y_pred_train_bay))
print('TRAIN : Mean Squared Error:', metrics.mean_squared_error(y_train, y_pred_train_bay))
print('TRAIN : Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train_bay)))
print ('----------------------------------------------------------')
#TEST:
print('TEST : Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_test_bay  ))
print('TEST : Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_test_bay  ))
print('TEST : Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_test_bay  )))

TRAIN : Mean Absolute Error: 843.7157383412579
TRAIN : Mean Squared Error: 1472223.007979303
TRAIN : Root Mean Squared Error: 1213.3519720094837
----------------------------------------------------------
TEST : Mean Absolute Error: 849.3952134449913
TEST : Mean Squared Error: 1570550.0493438118
TEST : Root Mean Squared Error: 1253.2158829761981


In [34]:
#Lets corroborate these values with cross validation:
scores_bay = cross_val_score(bay, X_train, y_train, cv=3, scoring='neg_root_mean_squared_error')
scores_bay

array([-1224.05516392, -1223.60517238, -1193.52355105])

In [35]:
#It is more than 1.000 --------> NOT VALID

## 5. Elastic net

In [36]:
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

In [37]:
elnet = ElasticNet(random_state=0)
elnet.fit(X_train, y_train)

ElasticNet(random_state=0)

In [38]:
y_pred_train_elnet = elnet.predict(X_train)
y_pred_test_elnet = elnet.predict(X_test)

In [39]:
#TRAIN:
print('TRAIN : Mean Absolute Error:', metrics.mean_absolute_error(y_train, y_pred_train_elnet))
print('TRAIN : Mean Squared Error:', metrics.mean_squared_error(y_train, y_pred_train_elnet))
print('TRAIN : Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train_elnet)))
print ('----------------------------------------------------------')
#TEST:
print('TEST : Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_test_elnet))
print('TEST : Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_test_elnet))
print('TEST : Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_test_elnet)))

TRAIN : Mean Absolute Error: 1293.9500213480783
TRAIN : Mean Squared Error: 3684598.3108922644
TRAIN : Root Mean Squared Error: 1919.5307527862803
----------------------------------------------------------
TEST : Mean Absolute Error: 1281.006185225804
TEST : Mean Squared Error: 3645784.000642738
TEST : Root Mean Squared Error: 1909.393621190439


In [40]:
#Lets corroborate these values with cross validation:
scores_elnet = cross_val_score(elnet, X_train, y_train, cv=3, scoring='neg_root_mean_squared_error')
scores_elnet

array([-1960.80804194, -1914.48020633, -1884.51142366])

In [41]:
#It is more than 1.000 --------> NOT VALID

# 6. Linear regression

In [69]:
#LINEAR REGRESSION
from sklearn.linear_model import LinearRegression

#Train the model
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [70]:
y_pred_train_lr = lr.predict(X_train)
y_pred_test_lr = lr.predict(X_test)

In [71]:
#TRAIN:
print('TRAIN : Mean Absolute Error:', metrics.mean_absolute_error(y_train, y_pred_train_lr))
print('TRAIN : Mean Squared Error:', metrics.mean_squared_error(y_train, y_pred_train_lr))
print('TRAIN : Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_train, y_pred_train_lr)))
print ('----------------------------------------------------------')
#TEST:
print('TEST : Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_test_lr))
print('TEST : Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_test_lr))
print('TEST : Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_test_lr)))

TRAIN : Mean Absolute Error: 843.7260406404276
TRAIN : Mean Squared Error: 1472223.0030218344
TRAIN : Root Mean Squared Error: 1213.3519699666022
----------------------------------------------------------
TEST : Mean Absolute Error: 849.4076390826287
TEST : Mean Squared Error: 1570559.973626702
TEST : Root Mean Squared Error: 1253.2198424964001


In [73]:
#Training set
r2_train_lr = r2_score(y_pred = y_pred_train_lr, y_true=y_train)
r2_train_lr

0.9083348287298033

In [74]:
#Training set
r2_test_lr = r2_score(y_pred = y_pred_test_lr, y_true=y_test)
r2_test_lr

0.9031247443748905

In [39]:
%store diamond

Stored 'diamond' (DataFrame)


## SECOND PHASE

In [39]:
diamond_1 = pd.read_csv("predict.csv", index_col="id") 

In [40]:
diamond_1 = diamond_1.drop(columns = ['x','y','z']) #HIGH CORRELATION
diamond_1["cut"].replace({"Ideal": 5, "Premium": 4, "Very Good" : 3, "Good" : 2, "Fair" : 1}, inplace=True)
diamond_1["color"].replace({"D": 7, "E": 6, "F" : 5, "G" : 4, "H" : 3, "I": 2 , "J": 1}, inplace=True)
diamond_1["clarity"].replace({"IF": 9, "VVS1": 8, "VVS2" : 7, 
                             "VS1" : 6, "VS2" : 5, "SI1": 4, 
                             "SI2" : 2, "I1": 1 }, inplace=True)

In [41]:
diamond_1.head()

,carat,cut,color,clarity,depth,table
id,,,,,,
0,0.45,4,6,4,62.8,58.0
1,1.23,5,3,4,61.0,56.0
2,0.33,5,2,9,61.8,55.0
3,0.51,4,7,5,58.0,60.0
4,0.40,4,6,5,62.2,59.0


In [42]:
y_pred_test_diamond_1 = randforest.predict(diamond_1)

In [43]:
diamond_1.head()

,carat,cut,color,clarity,depth,table
id,,,,,,
0,0.45,4,6,4,62.8,58.0
1,1.23,5,3,4,61.0,56.0
2,0.33,5,2,9,61.8,55.0
3,0.51,4,7,5,58.0,60.0
4,0.40,4,6,5,62.2,59.0


In [44]:
diamond_1["price"]=y_pred_test_diamond_1
diamond_1.head()

,carat,cut,color,clarity,depth,table,price
id,,,,,,,
0,0.45,4,6,4,62.8,58.0,15501.55
1,1.23,5,3,4,61.0,56.0,16874.20
2,0.33,5,2,9,61.8,55.0,8548.30
3,0.51,4,7,5,58.0,60.0,15268.37
4,0.40,4,6,5,62.2,59.0,8548.30


In [45]:
diamond_1["price"].to_csv("test_3.csv",header= True )